In [1]:
import yfinance as yf
import pandas as pd
import datetime as dt
import os

def pull_stock_data(tickers:str, start:str, end:str, interval:str, stripdateindex:str, column:str) -> pd.DataFrame:
    """
    Returns a dataframe with chosen stock information and treasury bill returns.

    Parameters:
    -------------
    tickers (str): Yahoo finance tickers for companies divided ONLY by single space
    start (str): start date of the period (format yyyy-mm-dd)
    end (str): end date of the period (format yyyy-mm-dd)
    interval (str): wanted interval (1d, 1m, 1y)
    stripdateindex (str): argument for stripping datetime index down (d - day, m - month, y- year)
    columns (list): list of wanted values, args same as in yahoo finance
    """
    # data download
    df = yf.download(tickers = tickers, start = start, end = end, interval = interval, groupby = 'ticker')
    # changing index from datetime to just year and month
    df.index = pd.to_datetime(df.index).to_period(stripdateindex)
    # dropping na rows
    df.dropna(inplace = True)
    return df[column]
    
def save_to_desktop(dataframe:pd.DataFrame, file_name:str):
    """
    Saves dataframe to desktop in csv format under filename provided

    Parameters:
    ------------
    dataframe (pd.DataFrame): dataframe that is to be saved
    file_name (str): name of the file
    """
    desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
    dataframe.to_csv(desktop+'/'+file_name+'.csv', sep=';', encoding='UTF-8')


In [3]:
tick = "NKE MSFT XOM INTC CAT WMT JPM F UPS MKC ^IRX"
st = '2000-12-01'
en = '2020-12-31'
intv = '1mo'
strpdt = 'm'
col = 'Adj Close'

dftest = pull_stock_data(tick, st, en, intv, strpdt, col)
dftest.head(20)

[*********************100%***********************]  11 of 11 completed


C:\Users\maxim\AppData\Local\Temp\ipykernel_3948\3181067758.py:22: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.index = pd.to_datetime(df.index).to_period(stripdateindex)


,CAT,F,INTC,JPM,MKC,MSFT,NKE,UPS,WMT,XOM,^IRX
Date,,,,,,,,,,,
2000-12,13.426955,12.510593,17.972366,23.754150,5.809331,13.602906,5.512078,33.709507,35.416012,21.869652,5.730
2001-01,12.549327,15.047401,22.119833,28.748075,5.904673,19.149918,5.440068,35.516922,37.913380,21.168447,4.840
2001-02,11.895667,15.004972,17.075621,24.571974,6.366356,18.503094,3.859768,32.435707,33.434528,20.388630,4.720
2001-03,12.690619,15.172241,15.739321,23.645109,6.802120,17.150646,3.990303,32.753658,33.708195,20.482904,4.180
2001-04,14.354871,15.906028,18.489395,25.267090,6.397516,21.247192,4.141493,33.070278,34.586437,22.404766,3.830
2001-05,15.596687,13.272593,16.156536,26.096521,6.576582,21.695658,4.071164,34.221535,34.593124,22.442684,3.540
2001-06,14.413118,13.381610,17.507399,23.611641,6.840295,22.893663,4.159321,33.378876,32.621151,22.197517,3.560
2001-07,15.867391,13.883076,17.842579,22.990416,6.982388,20.757963,4.716828,32.818695,37.420864,21.229015,3.440
2001-08,14.493443,10.830653,16.735283,21.076159,7.392926,17.891554,4.959860,31.894718,32.165909,20.410563,3.280


In [4]:
save_to_desktop(dftest, 'data')